Bi-montlhy Landsat -> https://browser.stac.dataspace.copernicus.eu/collections/opengeohub-landsat-bimonthly-mosaic-v1.0.1?.language=pt

# Extração

In [1]:
from pathlib import Path

BATCH_SIZE = 1_000_000

input_fn = "gpw_grassland_fscs.vi.vhr_grid.samples_20000101_20241231_go_epsg.4326_v2.gpkg"
input_dir = Path("../data")
output_dir = Path("../data/raw")
output_dir.mkdir(exist_ok=True)

In [2]:
import geopandas as gpd

map_dict = {
    "Other land cover": 0.0,
    "Cultivated grassland": 1.0,
    "Natural/semi-natural grassland": 2.0**6
}

processed = 0

while True:
    gdf = gpd.read_file(input_dir / input_fn, rows=slice(processed, processed + BATCH_SIZE))

    if gdf.empty:
        print("No more data to process.")
        break

    if gdf.shape[0] + 1 == BATCH_SIZE:
        last_tile_id = gdf.iloc[-1]["tile_id"]
    else:
        last_tile_id = None

    for tile_id in gdf["tile_id"].unique():
        if last_tile_id == tile_id:
            continue 

        tile_gdf = gdf[gdf["tile_id"] == tile_id].copy()

        tile_gdf["google_value"] = tile_gdf["google_class"].map(map_dict)

        tile_gdf.to_file(output_dir / f'{tile_id}.gpkg', driver="GPKG")

    processed += len(gdf)

    print(f"Processed up to rowid: {processed}")

Processed up to rowid: 1000000
Processed up to rowid: 2000000
Processed up to rowid: 3000000
Processed up to rowid: 4000000
Processed up to rowid: 5000000
Processed up to rowid: 6000000
Processed up to rowid: 7000000
Processed up to rowid: 8000000
Processed up to rowid: 9000000
Processed up to rowid: 10000000
Processed up to rowid: 11000000
Processed up to rowid: 12000000
Processed up to rowid: 13000000
Processed up to rowid: 14000000
Processed up to rowid: 15000000
Processed up to rowid: 16000000
Processed up to rowid: 17000000
Processed up to rowid: 18000000
Processed up to rowid: 19000000
Processed up to rowid: 20000000
Processed up to rowid: 21000000
Processed up to rowid: 22000000
Processed up to rowid: 23000000
Processed up to rowid: 24000000
Processed up to rowid: 25000000
Processed up to rowid: 26000000
Processed up to rowid: 27000000
Processed up to rowid: 28000000
Processed up to rowid: 29000000
Processed up to rowid: 30000000
Processed up to rowid: 31000000
Processed up to r

# Rasterizar

In [ ]:
# gdal_rasterize -a google_class -l 1-mvp -tr 10 10 -te 3298060 9564778 3299080 9565798 -ot Int16 -a_nodata -1 1-mvp.gpkg 1-mvp.tif

# gdalwarp -t_srs EPSG:32635 -te 624600 7192530 625060 7193000 -tr 10 10 -r mode -dstnodata -1 1-mvp.tif 1-mvp-warped.tif

In [1]:
import shutil
from pathlib import Path

input_dir = Path("../data/raw")
output_dir = Path("../data/raster")
output_dir.mkdir(exist_ok=True)

gpkg_files = list(input_dir.glob("*.gpkg"))

In [2]:
import rasterio
import numpy as np
import geopandas as gpd

from osgeo import gdal, osr

In [3]:
def rasterize(input_dir: Path, output_dir: Path):
    layer = input_dir.stem
    gdf = gpd.read_file(input_dir, layer=layer)
    xmin, ymin, xmax, ymax = gdf.total_bounds

    options = gdal.RasterizeOptions(
        attribute="google_value",
        layers=[layer],
        outputType=gdal.GDT_Int16,
        xRes=10,
        yRes=10,
        noData=-1,
        outputBounds=(xmin, ymin, xmax, ymax)
    )

    gdal.Rasterize(output_dir / f"{layer}.tif", input_dir, options=options)

In [4]:
def get_output_bounds(ds):
    gt = ds.GetGeoTransform()
    xmin = gt[0]
    ymin = gt[3]
    xres = gt[1]
    yres = gt[5]
    xsize = ds.RasterXSize
    ysize = ds.RasterYSize
    xmax = xmin + xres * xsize
    ymax = ymin + yres * ysize

    return (xmin, ymin, xmax, ymax)

def get_epsg_from_gpkg(ds) -> int:
    proj = ds.GetProjection()
    srs = osr.SpatialReference()
    srs.ImportFromWkt(proj)

    return srs.GetAttrValue("AUTHORITY", 1)

def warp(input_dir: Path, output_dir: Path):
    input_fn = input_dir.stem

    ds = gdal.Open(Path("../data/feature") / f'{input_fn}.tif')

    warp_options = gdal.WarpOptions(
        format="GTiff",
        dstSRS=f"EPSG:{get_epsg_from_gpkg(ds)}",
        outputBounds=get_output_bounds(ds),
        xRes=10, yRes=10,                  
        resampleAlg="average", #  mode              
        dstNodata=-1                       
    )

    gdal.Warp(output_dir / f'{input_fn}.tif', input_dir, options=warp_options)

In [5]:
def mask(input_dir: Path, output_dir: Path):
    fn = input_dir.stem

    with rasterio.open(input_dir) as msrc:
        mask = msrc.read()
        mask = mask[:, ::-1, :]

    with rasterio.open(Path("../data/feature") / f'{fn}.tif') as fsrc:
        data = fsrc.read()
        profile = fsrc.profile.copy()

    nodata = profile.get("nodata")

    masked_data = np.where(np.isin(mask, [0, 1, 64]), data, nodata)

    data_concat = np.concatenate([masked_data, mask], axis=0)

    profile.update(count=data_concat.shape[0])

    with rasterio.open(output_dir / f'{fn}.tif', "w", **profile) as dst:
        dst.write(data_concat)

In [6]:
rtemp_dir = Path("../data/temp_r")
rtemp_dir.mkdir(exist_ok=True)

wtemp_dir = Path("../data/temp_w")
wtemp_dir.mkdir(exist_ok=True)

for gpkg_file in gpkg_files:
    rasterize(gpkg_file, rtemp_dir)

    warp(rtemp_dir / f"{gpkg_file.stem}.tif", wtemp_dir)

    mask(wtemp_dir / f"{gpkg_file.stem}.tif", output_dir)

shutil.rmtree(rtemp_dir)
shutil.rmtree(wtemp_dir)

c:\Users\tiago\anaconda3\envs\gdal\Lib\site-packages\osgeo\gdal.py:330: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


# Samples

In [1]:
from pathlib import Path

input_dir = Path("../data/raster")
output_dir = Path("../data")
output_dir.mkdir(exist_ok=True)

raster_files = list(input_dir.glob("*.tif"))

In [2]:
import rasterio
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [3]:
def raster2gpkg(input_dir):
    with rasterio.open(input_dir) as src:
        data = src.read()
        transform = src.transform
        crs = src.crs
        nodata = src.nodata

    _, rows, cols = data.shape

    geoms = []
    records = []

    for i in range(rows):
        for j in range(cols):
            values = data[:, i, j]

            if any(values == nodata):
                continue

            x, y = transform * (j + 0.5, i + 0.5)

            geoms.append(Point(x, y))

            record = {f"B{b+1:02}": values[b] for b in range(64)}

            record["class"] = {0: 0, 1: 1, 64: 2}[values[64]]
            record["tile_id"] = input_dir.stem

            records.append(record)

    gdf = gpd.GeoDataFrame(records, geometry=geoms, crs=crs)

    return gdf if not gdf.empty else None

In [4]:
def reduce_samples(gdf: gpd.GeoDataFrame, frac: int):
    gdfs = []

    for value in [0, 1, 2]:
        new_gdf = gdf[gdf['class'] == value].copy()

        if not new_gdf.empty:
            new_gdf = new_gdf.sample(frac=frac, random_state=0).copy()

            gdfs.append(new_gdf)

    return gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)

In [5]:
gdfs = []

for raster_file in raster_files:
    gdf = raster2gpkg(raster_file)

    if not gdf is None:
        gdf = gdf.to_crs("EPSG:4326")

        gdf = reduce_samples(gdf, 0.20)
        
        gdfs.append(gdf) 

gdf_all = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)

gdf_all.to_file(output_dir / 'samples.gpkg', layer="all_points", driver="GPKG")

gdf_all.to_parquet(output_dir / "samples.pq", index=False)

In [6]:
import pandas as pd

df = pd.read_parquet('../data/samples.pq')

df['class'].value_counts()

class
0    7366959
2    1856736
1     685985
Name: count, dtype: int64

# Train/Valid

In [10]:
from pathlib import Path

input_dir = Path("../data/samples")
output_fn = "samples.gpkg"

In [ ]:
import random
import geopandas as gpd

gdf = gpd.read_file("../data/samples/samples.gpkg")

lista = list(gdf['tile_id'].unique())

qtd_valid = max(1, int(len(lista) * 0.2))

ids_valid = random.sample(lista, qtd_valid)

print("ids_valid:", ids_valid)

ids_valid: ['536-mvp', '153-mvp', '1433-mvp', '796-mvp', '130-mvp', '397-mvp', '1515-mvp', '1447-mvp', '1550-mvp', '148-mvp', '1659-mvp', '1317-mvp', '1836-mvp', '501-mvp', '1519-mvp', '773-mvp', '191-mvp', '1034-mvp', '602-mvp', '1584-mvp', '261-mvp', '1202-mvp', '444-mvp', '282-mvp', '1543-mvp', '871-mvp', '1705-mvp', '757-mvp', '1504-mvp', '396-mvp', '1238-mvp', '392-mvp', '1537-mvp', '1290-mvp', '339-mvp', '213-mvp', '1263-mvp', '745-mvp', '346-mvp', '118-mvp', '1715-mvp', '200-mvp', '421-mvp', '1538-mvp', '589-mvp', '164-mvp', '1465-mvp', '1288-mvp', '1830-mvp', '1594-mvp', '1846-mvp', '333-mvp', '604-mvp', '1143-mvp', '1645-mvp', '883-mvp', '884-mvp', '304-mvp', '1721-mvp', '1380-mvp', '156-mvp', '1432-mvp', '1613-mvp', '293-mvp', '701-mvp', '1027-mvp', '19-mvp', '109-mvp', '1565-mvp', '476-mvp', '580-mvp', '1475-mvp', '1829-mvp', '13-mvp', '778-mvp', '847-mvp', '1451-mvp', '188-mvp', '1384-mvp', '1526-mvp', '1456-mvp', '1474-mvp', '1151-mvp', '927-mvp', '353-mvp', '1013-mvp', '1

In [6]:
gdf["is_valid"] = gdf["tile_id"].isin(ids_valid)

In [12]:
gdf.to_file(input_dir / output_fn, layer="all_points", driver="GPKG")